<center><h1>ResNet: Cifar100</h1></center>

## Imports

In [0]:
from __future__ import division,print_function

%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
from tqdm import tqdm_notebook as tqdm

import random
import matplotlib.pyplot as plt
import math

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init as init
from torch.autograd import Variable, grad
from torchvision import datasets, transforms
from torch.nn.parameter import Parameter

import zero_shot_calculate_log as callog

import warnings
warnings.filterwarnings('ignore')

In [0]:
torch.cuda.set_device(1) #Select the GPU

## Model definition

In [0]:
def conv3x3(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)

def G_p(ob, p):
    temp = ob.detach()

    temp = temp**p
    temp = temp.reshape(temp.shape[0],temp.shape[1],-1)
    temp = ((torch.matmul(temp,temp.transpose(dim0=2,dim1=1)))).sum(dim=2)
    temp = (temp.sign()*torch.abs(temp)**(1/p)).reshape(temp.shape[0],-1)

    return temp
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(in_planes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )
    
    def forward(self, x):
        t = self.conv1(x)
        out = F.relu(self.bn1(t))
        torch_model.record(t)
        torch_model.record(out)
        t = self.conv2(out)
        out = self.bn2(self.conv2(out))
        torch_model.record(t)
        torch_model.record(out)
        t = self.shortcut(x)
        out += t
        torch_model.record(t)
        out = F.relu(out)
        torch_model.record(out)
        
        return out
    
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = conv3x3(3,64)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)
        
        self.collecting = False
    
    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        y = self.linear(out)
        return y
    
    def record(self, t):
        if self.collecting:
            self.gram_feats.append(t)
    
    def gram_feature_list(self,x):
        self.collecting = True
        self.gram_feats = []
        self.forward(x)
        self.collecting = False
        temp = self.gram_feats
        self.gram_feats = []
        return temp
    
    def load(self, path="results/Zero_Shot/Zero_Shot_cifar100_ResNet34_OECC_tune_epoch_9.pth"):
        tm = torch.load(path,map_location="cpu")        
        self.load_state_dict(tm)
    
    def get_min_max(self, data, power):
        mins = []
        maxs = []
        
        for i in range(0,len(data),128):
            batch = data[i:i+128].cuda()
            feat_list = self.gram_feature_list(batch)
            for L,feat_L in enumerate(feat_list):
                if L==len(mins):
                    mins.append([None]*len(power))
                    maxs.append([None]*len(power))
                
                for p,P in enumerate(power):
                    g_p = G_p(feat_L,P)
                    
                    current_min = g_p.min(dim=0,keepdim=True)[0]
                    current_max = g_p.max(dim=0,keepdim=True)[0]
                    
                    if mins[L][p] is None:
                        mins[L][p] = current_min
                        maxs[L][p] = current_max
                    else:
                        mins[L][p] = torch.min(current_min,mins[L][p])
                        maxs[L][p] = torch.max(current_max,maxs[L][p])
        
        return mins,maxs
    
    def get_deviations(self,data,power,mins,maxs):
        deviations = []
        
        for i in range(0,len(data),128):            
            batch = data[i:i+128].cuda()
            feat_list = self.gram_feature_list(batch)
            batch_deviations = []
            for L,feat_L in enumerate(feat_list):
                dev = 0
                for p,P in enumerate(power):
                    g_p = G_p(feat_L,P)
                    
                    dev +=  (F.relu(mins[L][p]-g_p)/torch.abs(mins[L][p]+10**-6)).sum(dim=1,keepdim=True)
                    dev +=  (F.relu(g_p-maxs[L][p])/torch.abs(maxs[L][p]+10**-6)).sum(dim=1,keepdim=True)
                batch_deviations.append(dev.cpu().detach().numpy())
            batch_deviations = np.concatenate(batch_deviations,axis=1)
            deviations.append(batch_deviations)
        deviations = np.concatenate(deviations,axis=0)
        
        return deviations

torch_model = ResNet(BasicBlock, [3,4,6,3], num_classes=100)
torch_model.load()
torch_model.cuda()
torch_model.params = list(torch_model.parameters())
torch_model.eval()
print("Done")    

Done


## Datasets

<b>In-distribution Datasets</b>

In [0]:
batch_size = 128
mean = np.array([[0.4914, 0.4822, 0.4465]]).T

std = np.array([[0.2023, 0.1994, 0.2010]]).T
normalize = transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))

transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
        
    ])

transform_test = transforms.Compose([
    transforms.CenterCrop(size=(32, 32)),
        transforms.ToTensor(),
        normalize
    ])

train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR100('data/cifar100-data', train=True, download=True,
                   transform=transform_train),
    batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR100('data/cifar100-data', train=False, transform=transform_test),
    batch_size=batch_size)


Files already downloaded and verified


In [0]:
data_train = list(torch.utils.data.DataLoader(
        datasets.CIFAR100('data/cifar100-data', train=True, download=True,
                       transform=transform_test),
        batch_size=1, shuffle=False))

Files already downloaded and verified


In [0]:
data = list(torch.utils.data.DataLoader(
    datasets.CIFAR100('data/cifar100-data', train=False, download=True,
                   transform=transform_test),
    batch_size=1, shuffle=False))

Files already downloaded and verified


In [0]:
torch_model.eval()
correct = 0
total = 0
for x,y in test_loader:
    x = x.cuda()
    y = y.numpy()
    correct += (y==np.argmax(torch_model(x).detach().cpu().numpy(),axis=1)).sum()
    total += y.shape[0]
print("Accuracy: ",correct/total)


Accuracy:  0.761


<b>Out-of-distribution Datasets</b>

In [0]:
svhn = list(torch.utils.data.DataLoader(
    datasets.SVHN('data/svhn-data', split="test", download=True,
                   transform=transform_test),
    batch_size=1, shuffle=True))

Using downloaded and verified file: data/svhn-data/test_32x32.mat


In [0]:
#isun = list(torch.utils.data.DataLoader(
    #datasets.ImageFolder("iSUN/",transform=transform_test),batch_size=1,shuffle=False))

In [0]:
lsun = list(torch.utils.data.DataLoader(
    datasets.ImageFolder("data/LSUN_resize/",transform=transform_test),batch_size=1,shuffle=True))

In [0]:
tinyimagenet = list(torch.utils.data.DataLoader(
    datasets.ImageFolder("data/Imagenet_resize/",transform=transform_test),batch_size=1,shuffle=True))

## Code for Detecting OODs

<b> Extract predictions for train and test data </b>

In [0]:
train_preds = []
train_confs = []
train_logits = []
for idx in range(0,len(data_train),128):
    batch = torch.squeeze(torch.stack([x[0] for x in data_train[idx:idx+128]]),dim=1).cuda()
    
    logits = torch_model(batch)
    confs = F.softmax(logits,dim=1).cpu().detach().numpy()
    preds = np.argmax(confs,axis=1)
    logits = (logits.cpu().detach().numpy())#**2)#.sum(axis=1)

    train_confs.extend(np.max(confs,axis=1))    
    train_preds.extend(preds)
    train_logits.extend(logits)
print("Done")

test_preds = []
test_confs = []
test_logits = []

for idx in range(0,len(data),128):
    batch = torch.squeeze(torch.stack([x[0] for x in data[idx:idx+128]]),dim=1).cuda()
    
    logits = torch_model(batch)
    confs = F.softmax(logits,dim=1).cpu().detach().numpy()
    preds = np.argmax(confs,axis=1)
    logits = (logits.cpu().detach().numpy())#**2)#.sum(axis=1)

    test_confs.extend(np.max(confs,axis=1))    
    test_preds.extend(preds)
    test_logits.extend(logits)
print("Done")

Done
Done


<b> Code for detecting OODs by identifying anomalies in correlations </b>

In [0]:
import zero_shot_calculate_log as callog
def detect(all_test_deviations,all_ood_deviations, verbose=True):
    average_results = {}
    for i in range(1,11):
        random.seed(i)
        
        validation_indices = random.sample(range(len(all_test_deviations)),int(0.1*len(all_test_deviations)))
        test_indices = sorted(list(set(range(len(all_test_deviations)))-set(validation_indices)))

        validation = all_test_deviations[validation_indices]
        test_deviations = all_test_deviations[test_indices]

        t95 = validation.mean(axis=0)+10**-7

        test_deviations = (test_deviations/t95[np.newaxis,:]).sum(axis=1)
        ood_deviations = (all_ood_deviations/t95[np.newaxis,:]).sum(axis=1)
        
        results = callog.compute_metric(-test_deviations,-ood_deviations)
        for m in results:
            average_results[m] = average_results.get(m,0)+results[m]
    
    for m in average_results:
        average_results[m] /= i
    if verbose:
        callog.print_results(average_results)
    return average_results

def Analyse(dataset,POWERS=[10]):
    ood_preds = []
    ood_confs = []
    ood_logits = []
    ood = dataset
    
    for idx in range(0,len(ood),128):
        batch = torch.squeeze(torch.stack([x[0] for x in ood[idx:idx+128]]),dim=1).cuda()
        logits = torch_model(batch)
        confs = F.softmax(logits,dim=1).cpu().detach().numpy()
        preds = np.argmax(confs,axis=1)
        logits = (logits.cpu().detach().numpy())#**2).sum(axis=1)

        ood_confs.extend(np.max(confs,axis=1))
        ood_preds.extend(preds)   
        ood_logits.extend(logits)
    print("Done")
    
    def compute_deviation(PRED):
        train_indices = np.where(np.array(train_preds)==PRED)[0]
        test_indices = np.where(np.array(test_preds)==PRED)[0]
        ood_indices = np.where(np.array(ood_preds)==PRED)[0]

        train_PRED = torch.squeeze(torch.stack([data_train[i][0] for i in train_indices]),dim=1)
        test_PRED = torch.squeeze(torch.stack([data[i][0] for i in test_indices]),dim=1)
        if len(ood_indices)!=0:
            ood_PRED = torch.squeeze(torch.stack([ood[i][0] for i in ood_indices]),dim=1)

        test_confs_PRED = np.array([test_confs[i] for i in test_indices])
        if len(ood_indices)!=0:
            ood_confs_PRED =  np.array([ood_confs[i] for i in ood_indices])

        mins,maxs = torch_model.get_min_max(train_PRED,power=POWERS)

        test_deviations = torch_model.get_deviations(test_PRED,power=POWERS,mins=mins,maxs=maxs)/test_confs_PRED[:,np.newaxis]

        if len(ood_indices)!=0:
            ood_deviations = torch_model.get_deviations(ood_PRED,power=POWERS,mins=mins,maxs=maxs)/ood_confs_PRED[:,np.newaxis]
        else:
            ood_deviations = None

        return test_deviations,ood_deviations
    
    CLASSES = range(100)
    
    all_test_deviations = None
    all_ood_deviations = None

    
    for PRED in tqdm(CLASSES):
        test_deviations,ood_deviations = compute_deviation(PRED)
        
        if all_test_deviations is not None:
            all_test_deviations = np.concatenate([all_test_deviations,test_deviations],axis=0) 
        else:
            all_test_deviations = test_deviations
        if ood_deviations is not None:
            if all_ood_deviations is not None:
                all_ood_deviations = np.concatenate([all_ood_deviations,ood_deviations],axis=0)
            else:
                all_ood_deviations = ood_deviations
    
    
    average_results = detect(all_test_deviations,all_ood_deviations)
    return average_results, all_test_deviations, all_ood_deviations


<center><h1> Results </h1></center>

## Combination of Pre- and Post-Training Methods

In [0]:
print("SVHN")
svhn_results=Analyse(svhn,POWERS=range(1,11))

print("LSUN_R")
lsun_results=Analyse(lsun,POWERS=range(1,11))

#print("iSUN_R")
#isun_results=Analyse(isun,POWERS=range(1,11))

print("TinyImageNet_R")
tinyimagenet_results=Analyse(tinyimagenet,POWERS=range(1,11))

SVHN
Done



 TNR    AUROC  DTACC  AUIN   AUOUT 
 87.203 97.078 91.855 92.984 98.680
LSUN_R
Done



 TNR    AUROC  DTACC  AUIN   AUOUT 
 98.224 99.280 96.816 99.351 98.991
TinyImageNet_R
Done



 TNR    AUROC  DTACC  AUIN   AUOUT 
 95.824 98.750 95.466 98.781 98.446
